In [4]:
import sys
import numpy as np
from scipy.spatial import ConvexHull
from scipy import sparse
from sklearn.manifold import MDS #If you want the scikit learn mds
import hictoolbox
from scipy import ndimage, misc,signal
import seaborn as sns

import pandas as pd
from pandas import Series
import cooler
from cooler.create import ArrayLoader

SyntaxError: invalid syntax (<ipython-input-4-0c80756b938b>, line 6)

In [45]:
def raw2cool(path,chr_name,R=25000):
    #path : for example path to chr9_25kb.RAWobserved
    #chr_name : for example 'chr9'
    #R : resolution
    
    HiCfilename=path
    
    A=np.loadtxt(HiCfilename)
    A=np.int_(A)
    
    chr_size=A[-1][0]
    
    A=np.concatenate((A,np.transpose(np.array([A[:,1],A[:,0],A[:,2]]))), axis=0)
    A = sparse.coo_matrix( (A[:,2], (A[:,0],A[:,1])))
    binned_map=hictoolbox.bin2d(A,R,R)
    
    labels = [chr_name] 
    s = Series(chr_size,index=labels)
    bins = cooler.binnify(s, R)
    bins.loc[len(bins)]=[chr_name,chr_size,chr_size] #add last row so that there is same number of rows as in binned matrix
    
    c = binned_map.tocoo()
    f = pd.DataFrame({'bin1_id': c.row, 'bin2_id': c.col, 'count': c.data})
    f=f.query('bin1_id < bin2_id and bin2_id<{}'.format(bins.shape[0]))
    #lastdiagonalbin=binned_map[int(chr_size/R),int(chr_size/R)]
    #f.loc[f.index[-1]]=[int(chr_size/R),int(chr_size/R),lastdiagonalbin] #add last row, i.e last bin on diagonal, 13153215 is pandas label of rows,

    cooler.create_cooler('{}.cool'.format(chr_name), bins, f)

In [46]:
HiCfilename='/Users/antoineszatkownik/Desktop/M2/MEET-EU/scripts/data/GM12878/25kb_resolution_intrachromosomal/chr9_25kb.RAWobserved'
raw2cool(HiCfilename,'chr9')

/var/folders/gc/y0f_kmk133b6n0f35k5ctjxc0000gn/T/ipykernel_2229/4241866825.py:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lastdiagonalbin=binned_map[np.int(chr_size/R),np.int(chr_size/R)]
